<a href="https://colab.research.google.com/github/tronglinux123/AI-Project/blob/main/SMOTE_and_XGBClassifier_XGBRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.cloud import bigquery
from google.colab import auth, drive
import os
# Mount to Google Drive if data on Drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/doan_demo')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/doan_demo'

In [ ]:
from typing import Callable
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import warnings
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import gc
warnings.filterwarnings('ignore')



In [ ]:
df = pd.read_csv('x_ray_and_features_data.csv')

diseases = ['pneumonia', 'tuberculosis', 'bronchitis', 'arthritis', 'fracture', 'lung_cancer', 'scoliosis', 'lung_infection']
mimic_feats = ['diabete', 'age', 'hiv', 'oxygen', 'heart_rate', 'temperature', 'hemoglobin', 'rbc', 'wbc', 'smoke', 'cough']
img_feats = [str(x) for x in range(18)]
models = [XGBClassifier]
# Keep the necessary data
data = df[img_feats+mimic_feats]
y = df["pneumonia"]
xtr, xte, ytr, yte = train_test_split(data, y, test_size=0.35)
xtr = xtr.reset_index(drop=True)
ytr = ytr.reset_index(drop=True)
xtr

,0,1,2,3,4,5,6,7,8,9,...,age,hiv,oxygen,heart_rate,temperature,hemoglobin,rbc,wbc,smoke,cough
0,0.974961,0.569951,-2.978841e-39,-2.884733,0.898215,-3.382279e-39,3.460620e-40,1.435536,-0.905821,2.730172e-39,...,67,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,2.554136,0.713316,-1.240317e-39,-2.222116,0.546541,-4.947448e-39,1.667570e-40,2.605252,0.915055,4.914345e-39,...,80,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,1.329126,-0.050889,1.488400e-39,-4.061300,0.213763,2.538787e-39,2.373699e-39,1.812233,-1.716044,2.129670e-39,...,59,0,NaN,102.0,98.0,11.7,4.23,10.4,0,0
3,2.025495,0.949495,2.283800e-40,-1.766144,0.781004,-9.380400e-40,1.587914e-39,2.386674,0.711350,3.631480e-39,...,75,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,-0.309173,-3.030663,8.608180e-40,-4.818019,-1.271249,1.243162e-39,6.791070e-40,-1.146392,-2.389468,-1.365914e-39,...,48,0,NaN,NaN,NaN,7.2,2.51,10.2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18898,1.360868,0.793227,-2.968295e-39,-2.290114,1.101779,-3.637045e-39,-1.430908e-39,1.464353,-0.097813,2.715459e-39,...,71,0,NaN,NaN,NaN,10.7,3.25,6.2,0,0
18899,0.563908,-1.033254,-3.213990e-40,-2.266787,-0.657203,-2.160656e-39,-1.166030e-40,0.309123,-0.944887,3.450000e-41,...,42,0,NaN,NaN,NaN,9.4,3.58,7.3,1,0
18900,3.156225,2.554653,-5.328758e-39,-1.308189,1.063684,-8.659490e-39,-1.207161e-39,3.721054,1.364351,4.112530e-39,...,94,0,NaN,121.0,97.5,8.8,3.18,6.9,0,0
18901,1.337338,0.493933,4.254450e-40,-3.952914,-0.282668,-1.614880e-40,-5.359080e-40,1.861630,-1.669540,1.983978e-39,...,77,0,NaN,132.0,98.4,8.0,2.64,15.2,0,0


In [ ]:
full_feats = img_feats + mimic_feats

sm = SMOTE(sampling_strategy=1, random_state=42)
xtr_img_sm, ytr_sm = sm.fit_resample(xtr[img_feats], ytr)
xtr_img_sm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.974961,0.569951,-2.978841e-39,-2.884733,0.898215,-3.382279e-39,3.460620e-40,1.435536,-0.905821,2.730172e-39,0.915577,-2.523156e-39,-1.776418e-39,-7.239282e-39,0.317744,0.422111,1.753502,0.183111
1,2.554136,0.713316,-1.240317e-39,-2.222116,0.546541,-4.947448e-39,1.667570e-40,2.605252,0.915055,4.914345e-39,1.013051,2.619276e-39,-1.346700e-39,-6.457944e-39,1.245167,0.754143,3.098097,0.293142
2,1.329126,-0.050889,1.488400e-39,-4.061300,0.213763,2.538787e-39,2.373699e-39,1.812233,-1.716044,2.129670e-39,0.117562,4.805750e-40,2.638246e-39,-1.218817e-39,-2.722615,-1.877393,1.800646,-1.306731
3,2.025495,0.949495,2.283800e-40,-1.766144,0.781004,-9.380400e-40,1.587914e-39,2.386674,0.711350,3.631480e-39,-0.468705,1.603091e-39,-2.087726e-39,-4.256975e-39,1.032192,0.232504,2.707501,-1.022179
4,-0.309173,-3.030663,8.608180e-40,-4.818019,-1.271249,1.243162e-39,6.791070e-40,-1.146392,-2.389468,-1.365914e-39,-2.442797,3.869708e-39,1.118465e-39,-7.925530e-40,-2.933270,-2.530570,-0.180635,-3.131578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,1.176968,0.738245,-1.523327e-39,-3.699118,1.871437,1.189982e-39,5.748803e-40,2.008556,0.011846,4.225268e-39,0.725671,-1.167555e-39,-1.229523e-39,-4.043115e-39,-0.656349,-1.078607,2.382250,-1.472100
27510,0.414783,-2.347019,7.479884e-40,-4.161855,-0.303353,-5.024300e-40,-3.743858e-41,-0.679886,-1.795180,1.640556e-39,-0.600611,3.492356e-39,1.257382e-39,-6.305898e-40,-2.354951,-1.887540,0.671248,-2.257215
27511,0.632910,-1.151829,6.649542e-40,-3.590209,-0.168075,1.157395e-39,1.327850e-39,0.545911,-1.401724,1.020396e-39,-0.251030,1.504787e-39,8.325645e-40,1.835239e-40,-1.647838,-1.424358,1.013455,-2.041020
27512,-0.448492,-1.114065,1.464355e-39,-2.363367,-0.809681,-3.184319e-39,2.600477e-39,-0.209671,-1.996028,1.429478e-39,-1.762503,8.843286e-40,-9.333473e-40,-5.588027e-39,-1.193676,0.130693,0.443542,-1.348765


In [ ]:
n_original = len(xtr)
n_total = len(xtr_img_sm)
n_synthetic = n_total - n_original
n_synthetic

8611

In [ ]:
xtr_final = pd.DataFrame()
xtr_final[img_feats] = xtr_img_sm
xtr_final.loc[:n_original-1, mimic_feats] = xtr[mimic_feats].values
xtr_final.loc[n_original:, mimic_feats] = np.nan
X_train_final = xtr_final
y_train_final = ytr_sm
X_train_final

,0,1,2,3,4,5,6,7,8,9,...,age,hiv,oxygen,heart_rate,temperature,hemoglobin,rbc,wbc,smoke,cough
0,0.974961,0.569951,-2.978841e-39,-2.884733,0.898215,-3.382279e-39,3.460620e-40,1.435536,-0.905821,2.730172e-39,...,67.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2.554136,0.713316,-1.240317e-39,-2.222116,0.546541,-4.947448e-39,1.667570e-40,2.605252,0.915055,4.914345e-39,...,80.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
2,1.329126,-0.050889,1.488400e-39,-4.061300,0.213763,2.538787e-39,2.373699e-39,1.812233,-1.716044,2.129670e-39,...,59.0,0.0,NaN,102.0,98.0,11.7,4.23,10.4,0.0,0.0
3,2.025495,0.949495,2.283800e-40,-1.766144,0.781004,-9.380400e-40,1.587914e-39,2.386674,0.711350,3.631480e-39,...,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,-0.309173,-3.030663,8.608180e-40,-4.818019,-1.271249,1.243162e-39,6.791070e-40,-1.146392,-2.389468,-1.365914e-39,...,48.0,0.0,NaN,NaN,NaN,7.2,2.51,10.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,1.176968,0.738245,-1.523327e-39,-3.699118,1.871437,1.189982e-39,5.748803e-40,2.008556,0.011846,4.225268e-39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27510,0.414783,-2.347019,7.479884e-40,-4.161855,-0.303353,-5.024300e-40,-3.743858e-41,-0.679886,-1.795180,1.640556e-39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27511,0.632910,-1.151829,6.649542e-40,-3.590209,-0.168075,1.157395e-39,1.327850e-39,0.545911,-1.401724,1.020396e-39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27512,-0.448492,-1.114065,1.464355e-39,-2.363367,-0.809681,-3.184319e-39,2.600477e-39,-0.209671,-1.996028,1.429478e-39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cat_feats = ['diabete', 'hiv', 'smoke', 'cough']
X_train_img = xtr[img_feats]
X_train_labels = xtr[cat_feats]
X_train_labels

,diabete,hiv,smoke,cough
0,1,1,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,1,0,0,0
...,...,...,...,...
18898,1,0,0,0
18899,1,0,0,0
18900,0,0,0,0
18901,1,0,0,0


In [ ]:
X_synth_img = X_train_final.loc[n_original:, img_feats]
X_synth_img

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
18903,0.666412,-0.549700,1.355347e-40,-2.739769,0.927468,-1.179377e-39,2.047388e-39,0.952527,-1.225722,3.661849e-39,-0.338399,4.181063e-40,-7.838054e-40,-2.946546e-39,-1.350341,-0.985571,1.357001,-0.991329
18904,1.428145,0.492131,3.884031e-39,-3.654594,-0.757470,2.659204e-39,2.994847e-39,2.134801,-1.762972,4.330324e-40,-0.681119,-1.606171e-39,3.294138e-39,-2.718952e-39,-2.448916,-1.515713,1.706238,-1.270616
18905,1.085005,0.263881,1.508186e-40,-3.398203,0.788507,3.216187e-40,1.656946e-39,1.839555,-1.289578,2.451167e-39,0.179066,1.458178e-40,1.380275e-39,-1.597404e-39,-2.254028,-1.815687,1.752349,-0.991140
18906,0.743950,-0.642910,1.668897e-39,-4.330216,-0.120592,1.986947e-39,9.116590e-40,1.346229,-2.140685,1.329956e-39,0.730320,-1.232447e-39,6.712450e-40,-1.644573e-39,-3.105835,-2.989472,1.072371,-0.875380
18907,1.373803,0.612825,7.539435e-40,-4.211323,1.136317,-1.928540e-40,-1.239708e-39,2.423393,-1.410945,4.449861e-39,2.115884,5.262431e-40,-1.007011e-39,-2.878661e-39,-2.644716,-1.893694,2.040332,0.111424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,1.176968,0.738245,-1.523327e-39,-3.699118,1.871437,1.189982e-39,5.748803e-40,2.008556,0.011846,4.225268e-39,0.725671,-1.167555e-39,-1.229523e-39,-4.043115e-39,-0.656349,-1.078607,2.382250,-1.472100
27510,0.414783,-2.347019,7.479884e-40,-4.161855,-0.303353,-5.024300e-40,-3.743858e-41,-0.679886,-1.795180,1.640556e-39,-0.600611,3.492356e-39,1.257382e-39,-6.305898e-40,-2.354951,-1.887540,0.671248,-2.257215
27511,0.632910,-1.151829,6.649542e-40,-3.590209,-0.168075,1.157395e-39,1.327850e-39,0.545911,-1.401724,1.020396e-39,-0.251030,1.504787e-39,8.325645e-40,1.835239e-40,-1.647838,-1.424358,1.013455,-2.041020
27512,-0.448492,-1.114065,1.464355e-39,-2.363367,-0.809681,-3.184319e-39,2.600477e-39,-0.209671,-1.996028,1.429478e-39,-1.762503,8.843286e-40,-9.333473e-40,-5.588027e-39,-1.193676,0.130693,0.443542,-1.348765


In [ ]:
results = {}

for col in cat_feats:
    print("="*50)
    print("Training label:", col)

    X_train, X_test, y_train, y_test = train_test_split(
        X_train_img,
        X_train_labels[col],
        test_size=0.2,
        random_state=42,
        stratify=X_train_labels[col]
    )


    model = XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"{col} accuracy:", acc)
    print(f"{col} F1-score:", f1)
    print(f"{col} ROC-AUC:", auc)
    print(classification_report(y_test, y_pred))

    results[col] = {
        "accuracy": acc,
        "f1": f1,
        "auc": auc
    }

Training label: diabete
Class balance: 0=7832, 1=7290, scale_pos_weight=1.074
diabete accuracy: 0.5807987304945782
diabete F1-score: 0.5754085186177337
diabete ROC-AUC: 0.6130562996822466
              precision    recall  f1-score   support

           0       0.60      0.57      0.59      1958
           1       0.56      0.59      0.58      1823

    accuracy                           0.58      3781
   macro avg       0.58      0.58      0.58      3781
weighted avg       0.58      0.58      0.58      3781

Training label: hiv
Class balance: 0=14542, 1=580, scale_pos_weight=25.072
hiv accuracy: 0.7884157630256546
hiv F1-score: 0.12280701754385964
hiv ROC-AUC: 0.6438697318007662
              precision    recall  f1-score   support

           0       0.97      0.80      0.88      3636
           1       0.07      0.39      0.12       145

    accuracy                           0.79      3781
   macro avg       0.52      0.60      0.50      3781
weighted avg       0.94      0.79      

In [ ]:
models_cat = {}
for col in cat_feats:
    mask = X_train_labels[col]
    n_pos = mask.sum()
    n_neg = len(mask) - n_pos
    model = XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight = n_neg / n_pos
    )

    model.fit(X_train_img, X_train_labels[col])
    models_cat[col] = model

preds = {}

for col in cat_feats:
    preds[col] = models_cat[col].predict(X_synth_img)

for col in cat_feats:
    X_train_final.loc[n_original:, col] = preds[col]

In [ ]:
import pandas as pd

summary = {}

for col in cat_feats:
    counts = X_train_final.loc[:n_original-1, col].value_counts()
    summary[col] = {
        "0_count": counts.get(0, 0),
        "1_count": counts.get(1, 0)
    }

df_summary = pd.DataFrame(summary).T
print(df_summary)


         0_count  1_count
diabete     9794     9109
hiv        18193      710
smoke      16624     2279
cough      18863       40


In [ ]:
import pandas as pd

summary = {}

for col in cat_feats:
    counts = X_train_final.loc[n_original:, col].value_counts()
    summary[col] = {
        "0_count": counts.get(0, 0),
        "1_count": counts.get(1, 0)
    }

df_summary = pd.DataFrame(summary).T
print(df_summary)


         0_count  1_count
diabete     4408     4203
hiv         6632     1979
smoke       5606     3005
cough       8537       74


In [ ]:
X_train_final

,0,1,2,3,4,5,6,7,8,9,...,age,hiv,oxygen,heart_rate,temperature,hemoglobin,rbc,wbc,smoke,cough
0,0.974961,0.569951,-2.978841e-39,-2.884733,0.898215,-3.382279e-39,3.460620e-40,1.435536,-0.905821,2.730172e-39,...,67.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2.554136,0.713316,-1.240317e-39,-2.222116,0.546541,-4.947448e-39,1.667570e-40,2.605252,0.915055,4.914345e-39,...,80.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
2,1.329126,-0.050889,1.488400e-39,-4.061300,0.213763,2.538787e-39,2.373699e-39,1.812233,-1.716044,2.129670e-39,...,59.0,0.0,NaN,102.0,98.0,11.7,4.23,10.4,0.0,0.0
3,2.025495,0.949495,2.283800e-40,-1.766144,0.781004,-9.380400e-40,1.587914e-39,2.386674,0.711350,3.631480e-39,...,75.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,-0.309173,-3.030663,8.608180e-40,-4.818019,-1.271249,1.243162e-39,6.791070e-40,-1.146392,-2.389468,-1.365914e-39,...,48.0,0.0,NaN,NaN,NaN,7.2,2.51,10.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,1.176968,0.738245,-1.523327e-39,-3.699118,1.871437,1.189982e-39,5.748803e-40,2.008556,0.011846,4.225268e-39,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
27510,0.414783,-2.347019,7.479884e-40,-4.161855,-0.303353,-5.024300e-40,-3.743858e-41,-0.679886,-1.795180,1.640556e-39,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
27511,0.632910,-1.151829,6.649542e-40,-3.590209,-0.168075,1.157395e-39,1.327850e-39,0.545911,-1.401724,1.020396e-39,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
27512,-0.448492,-1.114065,1.464355e-39,-2.363367,-0.809681,-3.184319e-39,2.600477e-39,-0.209671,-1.996028,1.429478e-39,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
mimic_numeric = ['oxygen', 'heart_rate', 'temperature',
                   'hemoglobin', 'rbc', 'wbc']

models_mask = {}
models_value = {}

In [ ]:
results_mask = {}

for col in mimic_numeric:
    print("="*50)
    print("Training label:", col)
    mask = xtr[col].notna().astype(int)
    print(mask.value_counts())
    X_train_num_input = pd.concat([X_train_img, X_train_labels], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
        X_train_num_input,
        mask,
        test_size=0.2,
        random_state=42,
    )

    model = XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"{col} accuracy:", acc)
    print(f"{col} F1-score:", f1)
    print(f"{col} ROC-AUC:", auc)
    print(classification_report(y_test, y_pred))

    results_mask[col] = {"accuracy": acc, "f1": f1, "auc": auc}

    value_model = XGBRegressor(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    value_model.fit(X_train_num_input[mask == 1], xtr.loc[mask == 1, col])
    models_value[col] = value_model


Training label: oxygen
oxygen
0    17721
1     1182
Name: count, dtype: int64
oxygen accuracy: 0.9441946574980163
oxygen F1-score: 0.018604651162790697
oxygen ROC-AUC: 0.6446842235570194
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3573
           1       0.29      0.01      0.02       208

    accuracy                           0.94      3781
   macro avg       0.62      0.50      0.49      3781
weighted avg       0.91      0.94      0.92      3781

Training label: heart_rate
heart_rate
0    14289
1     4614
Name: count, dtype: int64
heart_rate accuracy: 0.7664638984395663
heart_rate F1-score: 0.0849740932642487
heart_rate ROC-AUC: 0.647679369626607
              precision    recall  f1-score   support

           0       0.77      0.98      0.87      2904
           1       0.47      0.05      0.08       877

    accuracy                           0.77      3781
   macro avg       0.62      0.52      0.48      3781
weighted a

In [ ]:
mimic_age = ['diabete', 'hiv', 'oxygen', 'heart_rate', 'temperature', 'hemoglobin', 'rbc', 'wbc', 'smoke', 'cough']
X_train_age = xtr[img_feats + mimic_age]
y_train_age = xtr['age']
age_model = XGBRegressor(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

age_model.fit(X_train_age, y_train_age)
X_synth_age_input = X_train_final.loc[n_original:, img_feats + mimic_age]
age_pred = age_model.predict(X_synth_age_input)

X_train_final.loc[n_original:, 'age'] = age_pred

,age
0,46
1,77
2,67
3,56
4,79
...,...
18898,56
18899,69
18900,50
18901,76


In [ ]:
mimic_numeric = ['oxygen', 'heart_rate', 'temperature',
                   'hemoglobin', 'rbc', 'wbc']
models_mask = {}
models_value = {}

for num in mimic_numeric:
    mask = xtr[num].notna().astype(int)
    n_pos = mask.sum()
    n_neg = len(mask) - n_pos

    X_train_num_input = pd.concat([X_train_img, X_train_labels], axis=1)

    mask_model = XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight = n_neg / n_pos
    )

    mask_model.fit(X_train_num_input, mask)
    models_mask[num] = mask_model

    value_model = XGBRegressor(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    value_model.fit(X_train_num_input[mask == 1], xtr.loc[mask == 1, num])
    models_value[num] = value_model

mimic_age = ['diabete', 'hiv', 'oxygen', 'heart_rate', 'temperature', 'hemoglobin', 'rbc', 'wbc', 'smoke', 'cough']
X_train_age = xtr[img_feats + mimic_age]
y_train_age = xtr['age']
age_model = XGBRegressor(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

age_model.fit(X_train_age, y_train_age)
X_synth_age_input = X_train_final.loc[n_original:, img_feats + mimic_age]
age_pred = age_model.predict(X_synth_age_input)

X_train_final.loc[n_original:, 'age'] = age_pred

X_synth_input = pd.concat([X_synth_img, X_train_final.loc[n_original:, cat_feats]], axis=1)

for num in mimic_numeric:
    mask_pred = models_mask[num].predict(X_synth_input)
    idx_has_value = (mask_pred == 1)

    if idx_has_value.sum() > 0:
        values = models_value[num].predict(X_synth_input[idx_has_value])
        X_train_final.loc[n_original:].loc[idx_has_value, num] = values
X_train_final

,0,1,2,3,4,5,6,7,8,9,...,age,hiv,oxygen,heart_rate,temperature,hemoglobin,rbc,wbc,smoke,cough
0,0.974961,0.569951,-2.978841e-39,-2.884733,0.898215,-3.382279e-39,3.460620e-40,1.435536,-0.905821,2.730172e-39,...,67.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2.554136,0.713316,-1.240317e-39,-2.222116,0.546541,-4.947448e-39,1.667570e-40,2.605252,0.915055,4.914345e-39,...,80.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
2,1.329126,-0.050889,1.488400e-39,-4.061300,0.213763,2.538787e-39,2.373699e-39,1.812233,-1.716044,2.129670e-39,...,59.000000,0.0,NaN,102.000000,98.000000,11.700000,4.230000,10.400000,0.0,0.0
3,2.025495,0.949495,2.283800e-40,-1.766144,0.781004,-9.380400e-40,1.587914e-39,2.386674,0.711350,3.631480e-39,...,75.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,-0.309173,-3.030663,8.608180e-40,-4.818019,-1.271249,1.243162e-39,6.791070e-40,-1.146392,-2.389468,-1.365914e-39,...,48.000000,0.0,NaN,NaN,NaN,7.200000,2.510000,10.200000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,1.176968,0.738245,-1.523327e-39,-3.699118,1.871437,1.189982e-39,5.748803e-40,2.008556,0.011846,4.225268e-39,...,64.822128,0.0,NaN,75.833725,149.204758,NaN,NaN,NaN,0.0,0.0
27510,0.414783,-2.347019,7.479884e-40,-4.161855,-0.303353,-5.024300e-40,-3.743858e-41,-0.679886,-1.795180,1.640556e-39,...,68.910156,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
27511,0.632910,-1.151829,6.649542e-40,-3.590209,-0.168075,1.157395e-39,1.327850e-39,0.545911,-1.401724,1.020396e-39,...,67.828300,0.0,NaN,84.997620,97.978279,10.113045,3.430074,8.612871,0.0,0.0
27512,-0.448492,-1.114065,1.464355e-39,-2.363367,-0.809681,-3.184319e-39,2.600477e-39,-0.209671,-1.996028,1.429478e-39,...,69.921013,0.0,NaN,NaN,NaN,10.539726,3.523836,8.358151,0.0,0.0
